<a href="https://colab.research.google.com/github/mithlohar144/Air_Quality_Prediction_System/blob/main/day19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Perform time series forecasting using multiple models, evaluate their performance, and identify the best model for forecasting.

## Data preprocessing

### Subtask:
Ensure the time series data is in the correct format for forecasting models, handling missing values and potential outliers.


**Reasoning**:
Examine the `so2_data` Series for missing values and their extent. Handle missing values using interpolation and verify the result. Outlier detection and handling will be considered in a later step if needed.



In [ ]:
import pandas as pd
import numpy as np

# Check for missing values and their extent
missing_values_count = so2_data.isnull().sum()
print(f"Number of missing values before handling: {missing_values_count}")

# Handle missing values using interpolation (linear method is a common choice for time series)
so2_data_filled = so2_data.interpolate(method='linear')

# Verify that missing values have been handled
missing_values_count_after = so2_data_filled.isnull().sum()
print(f"Number of missing values after handling: {missing_values_count_after}")

# Display the first few rows of the handled data
display(so2_data_filled.head())

# Display the last few rows of the handled data
display(so2_data_filled.tail())


Number of missing values before handling: 15
Number of missing values after handling: 15


,so2
sampling_date,
1987-03-09,NaN
1987-03-10,NaN
1987-03-11,NaN
1987-03-12,NaN
1987-03-13,NaN


,so2
sampling_date,
2015-12-27,9.000000
2015-12-28,6.916667
2015-12-29,8.076923
2015-12-30,9.142857
2015-12-31,8.625000


**Reasoning**:
Since there are still missing values at the beginning of the series after linear interpolation, fill these remaining leading NaNs using forward fill. Then, check for outliers using a common method like the Z-score and address any identified outliers by capping them.



In [ ]:
# Fill any remaining leading NaNs using forward fill
so2_data_filled = so2_data_filled.fillna(method='bfill')

# Verify that all missing values have been handled
missing_values_count_after_bfill = so2_data_filled.isnull().sum()
print(f"Number of missing values after backward fill: {missing_values_count_after_bfill}")

# Check for outliers using Z-score
z_scores = np.abs((so2_data_filled - so2_data_filled.mean()) / so2_data_filled.std())
outliers = so2_data_filled[z_scores > 3]
print(f"Number of potential outliers identified by Z-score > 3: {len(outliers)}")

# Address outliers by capping values exceeding a certain Z-score threshold (e.g., 3)
so2_data_cleaned = so2_data_filled.copy()
threshold = 3
capped_value_upper = so2_data_cleaned.mean() + threshold * so2_data_cleaned.std()
capped_value_lower = so2_data_cleaned.mean() - threshold * so2_data_cleaned.std()
so2_data_cleaned = np.where(so2_data_cleaned > capped_value_upper, capped_value_upper, so2_data_cleaned)
so2_data_cleaned = np.where(so2_data_cleaned < capped_value_lower, capped_value_lower, so2_data_cleaned)

# Convert the cleaned data back to a pandas Series with the original index
so2_data_cleaned = pd.Series(so2_data_cleaned, index=so2_data_filled.index)

# Display the first few rows of the cleaned data
display(so2_data_cleaned.head())

# Display the last few rows of the cleaned data
display(so2_data_cleaned.tail())


Number of missing values after backward fill: 0
Number of potential outliers identified by Z-score > 3: 290


/tmp/ipython-input-3958724463.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  so2_data_filled = so2_data_filled.fillna(method='bfill')


,0
sampling_date,
1987-03-09,37.339965
1987-03-10,37.339965
1987-03-11,37.339965
1987-03-12,37.339965
1987-03-13,37.339965


,0
sampling_date,
2015-12-27,9.000000
2015-12-28,6.916667
2015-12-29,8.076923
2015-12-30,9.142857
2015-12-31,8.625000


## Feature engineering

### Subtask:
Create relevant features from the time series data, such as lagged values or moving averages, that can improve model performance.


**Reasoning**:
Create lagged and moving average features, add the original series as the target, handle missing values, and display the resulting DataFrame.



**Reasoning**:
The previous code failed because the column name was not 'so2' in the dataframe created from the series. I need to check the column name and use the correct one to create lagged features.



## Model training and evaluation

### Subtask:
Train the selected models (ARIMA, Gradient Boosting, and LSTM) on the prepared data and evaluate their performance using appropriate metrics.

## Model selection

### Subtask:
Choose several time series forecasting models to evaluate, such as ARIMA, Prophet, or machine learning models like LSTMs.

**Reasoning**:
I need to select and list at least three distinct time series forecasting model types to evaluate, providing a brief justification for each.

In [ ]:
# 1. Statistical Model: ARIMA (AutoRegressive Integrated Moving Average)
# Justification: ARIMA models are well-established for time series forecasting.
# They capture temporal dependencies, trends, and seasonality through
# autoregressive, integrated, and moving average components. They are suitable
# for stationary or near-stationary time series data.

# 2. Machine Learning Model: Gradient Boosting Regressor (e.g., LightGBM or XGBoost)
# Justification: Tree-based models like Gradient Boosting can capture complex,
# non-linear relationships between the engineered features (lagged values,
# moving averages) and the target variable. They are generally robust to
# outliers and can handle a variety of data patterns.

# 3. Deep Learning Model: LSTM (Long Short-Term Memory)
# Justification: LSTMs are a type of recurrent neural network (RNN)
# particularly well-suited for sequence prediction tasks like time series
# forecasting. They can learn long-term dependencies in the data, making them
# effective for capturing intricate patterns and trends over time.

print("Selected Time Series Forecasting Models:")
print("- ARIMA (Statistical Model)")
print("- Gradient Boosting Regressor (Machine Learning Model)")
print("- LSTM (Deep Learning Model)")

Selected Time Series Forecasting Models:
- ARIMA (Statistical Model)
- Gradient Boosting Regressor (Machine Learning Model)
- LSTM (Deep Learning Model)


In [ ]:
# Create lagged features
df_features = pd.DataFrame(so2_data_cleaned)
# Check the column name of the dataframe
print(df_features.columns)

# Use the correct column name to create lagged features
column_name = df_features.columns[0] # Assuming the first column is the data
for i in range(1, 31):
    df_features[f'so2_lag_{i}'] = df_features[column_name].shift(i)

# Create moving average features
df_features[f'{column_name}_rolling_mean_7'] = df_features[column_name].rolling(window=7).mean()
df_features[f'{column_name}_rolling_mean_30'] = df_features[column_name].rolling(window=30).mean()


# Rename the original so2 column to be the target variable
df_features = df_features.rename(columns={column_name: 'so2_target'})

# Handle missing values introduced by lagging and rolling means
df_features.dropna(inplace=True)

# Display the first few rows of the df_features DataFrame
display(df_features.head())

RangeIndex(start=0, stop=1, step=1)


,so2_target,so2_lag_1,so2_lag_2,so2_lag_3,so2_lag_4,so2_lag_5,so2_lag_6,so2_lag_7,so2_lag_8,so2_lag_9,...,so2_lag_23,so2_lag_24,so2_lag_25,so2_lag_26,so2_lag_27,so2_lag_28,so2_lag_29,so2_lag_30,0_rolling_mean_7,0_rolling_mean_30
sampling_date,,,,,,,,,,,,,,,,,,,,,
1987-04-08,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,...,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965
1987-04-09,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,...,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965
1987-04-10,23.600000,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,...,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,35.377113,36.881966
1987-04-11,23.600000,23.600000,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,...,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,33.414261,36.423967
1987-04-12,23.600000,23.600000,23.600000,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,...,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,37.339965,31.451409,35.965968


## Model selection

### Subtask:
Choose several time series forecasting models to evaluate, such as ARIMA, Prophet, or machine learning models like LSTMs.


**Reasoning**:
I need to select and list at least three distinct time series forecasting model types to evaluate, providing a brief justification for each.



In [ ]:
# 1. Statistical Model: ARIMA (AutoRegressive Integrated Moving Average)
# Justification: ARIMA models are well-established for time series forecasting.
# They capture temporal dependencies, trends, and seasonality through
# autoregressive, integrated, and moving average components. They are suitable
# for stationary or near-stationary time series data.

# 2. Machine Learning Model: Gradient Boosting Regressor (e.g., LightGBM or XGBoost)
# Justification: Tree-based models like Gradient Boosting can capture complex,
# non-linear relationships between the engineered features (lagged values,
# moving averages) and the target variable. They are generally robust to
# outliers and can handle a variety of data patterns.

# 3. Deep Learning Model: LSTM (Long Short-Term Memory)
# Justification: LSTMs are a type of recurrent neural network (RNN)
# particularly well-suited for sequence prediction tasks like time series
# forecasting. They can learn long-term dependencies in the data, making them
# effective for capturing intricate patterns and trends over time.

print("Selected Time Series Forecasting Models:")
print("- ARIMA (Statistical Model)")
print("- Gradient Boosting Regressor (Machine Learning Model)")
print("- LSTM (Deep Learning Model)")

Selected Time Series Forecasting Models:
- ARIMA (Statistical Model)
- Gradient Boosting Regressor (Machine Learning Model)
- LSTM (Deep Learning Model)


## Model training and evaluation

### Subtask:
Train the selected models (ARIMA, Gradient Boosting, and LSTM) on the prepared data and evaluate their performance using appropriate metrics.


**Reasoning**:
Split the data, train the ARIMA model, make predictions, train the Gradient Boosting Regressor, make predictions, prepare data for LSTM, build, train and predict with LSTM, then evaluate all models.



In [ ]:
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


# 1. Split data into training and testing sets (time-based split)
train_size = int(len(df_features) * 0.8)
train_data, test_data = df_features[0:train_size], df_features[train_size:len(df_features)]

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

# 2. ARIMA model
# Using auto_arima or ACF/PACF for optimal orders is ideal, but for simplicity
# and demonstration, we'll choose a common order (p, d, q).
# A common starting point for non-seasonal ARIMA is (5,1,0) or (1,1,0)
# For demonstration, let's use (5,1,0)
try:
    arima_model = ARIMA(train_data['so2_target'], order=(5, 1, 0))
    arima_model_fit = arima_model.fit()
    arima_predictions = arima_model_fit.predict(start=len(train_data), end=len(df_features)-1)
except Exception as e:
    print(f"ARIMA model training or prediction failed: {e}")
    arima_predictions = pd.Series([np.nan] * len(test_data), index=test_data.index)


# 3. Gradient Boosting Regressor (LightGBM)
X_train, y_train = train_data.drop('so2_target', axis=1), train_data['so2_target']
X_test, y_test = test_data.drop('so2_target', axis=1), test_data['so2_target']

lgbm = lgb.LGBMRegressor(random_state=42)
lgbm.fit(X_train, y_train)
lgbm_predictions = lgbm.predict(X_test)

# 4. LSTM model
# Reshape data for LSTM: [samples, time steps, features]
# We'll use a lookback period (e.g., 30 days, as used for lagged features) as time steps
lookback = 30

def create_lstm_dataset(X, y, lookback=1):
    Xs, ys = [], []
    for i in range(len(X) - lookback):
        v = X.iloc[i:(i + lookback)].values
        Xs.append(v)
        ys.append(y.iloc[i + lookback])
    return np.array(Xs), np.array(ys)

# Scale data for LSTM
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))


X_train_lstm, y_train_lstm = create_lstm_dataset(pd.DataFrame(X_train_scaled), pd.Series(y_train_scaled.flatten()), lookback)
X_test_lstm, y_test_lstm = create_lstm_dataset(pd.DataFrame(X_test_scaled), pd.Series(y_test_scaled.flatten()), lookback)


# Build LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

# Train LSTM model with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Adjusting validation split size to be smaller if test set is already small after lookback
# Calculate available samples after lookback
available_train_samples = X_train_lstm.shape[0]
# Determine validation split ratio - aim for a reasonable number of validation samples
validation_split_ratio = 0.2 # Example ratio
if available_train_samples * validation_split_ratio < 50: # Arbitrary minimum samples for validation
    validation_split_ratio = min(0.5, 50 / available_train_samples) # Adjust if needed

history = lstm_model.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=32, validation_split=validation_split_ratio, verbose=0, callbacks=[early_stop])


# Make LSTM predictions
lstm_predictions_scaled = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler_y.inverse_transform(lstm_predictions_scaled).flatten()

# Adjust LSTM predictions index to match the corresponding test data points after lookback
lstm_predictions = pd.Series(lstm_predictions, index=test_data.index[lookback:])


# 5. Evaluate models
def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{model_name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}")
    return {'RMSE': rmse, 'MAE': mae}

evaluation_results = {}

# Evaluate ARIMA - need to align indices
arima_predictions_aligned = arima_predictions[test_data.index] # Align ARIMA predictions with test data index
evaluation_results['ARIMA'] = evaluate_model(test_data['so2_target'], arima_predictions_aligned, 'ARIMA')


# Evaluate LightGBM
evaluation_results['LightGBM'] = evaluate_model(y_test, lgbm_predictions, 'LightGBM')

# Evaluate LSTM - need to align indices
# The LSTM predictions start after the lookback period in the test set
evaluation_results['LSTM'] = evaluate_model(test_data['so2_target'][lookback:], lstm_predictions, 'LSTM')


# 6. Print evaluation metrics
print("\n--- Model Evaluation Results ---")
for model, metrics in evaluation_results.items():
    print(f"{model}:")
    print(f"  RMSE: {metrics['RMSE']:.4f}")
    print(f"  MAE: {metrics['MAE']:.4f}")


Training data shape: (8396, 33)
Testing data shape: (2099, 33)


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001917 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7216
[LightGBM] [Info] Number of data points in the train set: 8396, number of used features: 32
[LightGBM] [Info] Start training from score 15.463131


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
ARIMA - RMSE: 2.6056, MAE: 1.5342
LightGBM - RMSE: 1.6567, MAE: 1.0131
LSTM - RMSE: 2.2697, MAE: 1.3409

--- Model Evaluation Results ---
ARIMA:
  RMSE: 2.6056
  MAE: 1.5342
LightGBM:
  RMSE: 1.6567
  MAE: 1.0131
LSTM:
  RMSE: 2.2697
  MAE: 1.3409


## Model comparison

### Subtask:
Compare the performance of the different models to identify the best-performing one for forecasting.


**Reasoning**:
Access the evaluation_results dictionary to find the model with the lowest RMSE and MAE, and then print the best model and its metrics.



In [ ]:
# Determine the best performing model
best_model_name = None
lowest_rmse = float('inf')
lowest_mae = float('inf')

for model_name, metrics in evaluation_results.items():
    if metrics['RMSE'] < lowest_rmse:
        lowest_rmse = metrics['RMSE']
        best_model_name = model_name

# Find the MAE for the best model based on lowest RMSE
best_model_mae = evaluation_results[best_model_name]['MAE']


# Print the best model and its metrics
print(f"The best performing model is {best_model_name} with:")
print(f"  RMSE: {lowest_rmse:.4f}")
print(f"  MAE: {best_model_mae:.4f}")

The best performing model is LightGBM with:
  RMSE: 1.6567
  MAE: 1.0131


## Forecasting

### Subtask:
Use the best-performing model (LightGBM) to make future air quality forecasts on the test dataset.


**Reasoning**:
Generate future forecasts using the best-performing model (LightGBM) on the test dataset and store them in a pandas Series with the same index as y_test.



In [ ]:
# Use the trained LightGBM model to make predictions on the test dataset
final_forecasts = lgbm.predict(X_test)

# Convert the predictions to a pandas Series with the same index as y_test
final_forecasts = pd.Series(final_forecasts, index=y_test.index)

# Display the first few rows of the final forecasts
display(final_forecasts.head())

# Display the last few rows of the final forecasts
display(final_forecasts.tail())

,0
sampling_date,
2010-04-03,5.640353
2010-04-04,6.294726
2010-04-05,6.639475
2010-04-06,5.938009
2010-04-07,7.227837


,0
sampling_date,
2015-12-27,7.620390
2015-12-28,7.743291
2015-12-29,7.678225
2015-12-30,7.666843
2015-12-31,7.360038


## Summary:

### Data Analysis Key Findings

*   The initial dataset had 15 missing values, which were successfully handled using a combination of linear interpolation and backward fill.
*   290 potential outliers were identified using the Z-score method and addressed by capping values within a 3 standard deviation threshold.
*   Feature engineering involved creating 30 lagged features and two moving average features (7-day and 30-day rolling means).
*   Three types of models were selected for forecasting: ARIMA (Statistical), Gradient Boosting (Machine Learning), and LSTM (Deep Learning).
*   The data was split into 80% for training (8396 samples) and 20% for testing (2099 samples) using a time-based split.
*   Model evaluation on the test set showed the following performance metrics:
    *   ARIMA: RMSE: 2.6056, MAE: 1.5342
    *   LightGBM: RMSE: 1.6567, MAE: 1.0131
    *   LSTM: RMSE: 2.2697, MAE: 1.3409
*   LightGBM demonstrated the best performance with the lowest RMSE (1.6567) and MAE (1.0131).
*   The LightGBM model was successfully used to generate forecasts on the test dataset.

### Insights or Next Steps

*   LightGBM is the most suitable model among the evaluated options for forecasting this specific time series data, likely due to its ability to effectively utilize the engineered features.
*   Further refinement could involve hyperparameter tuning for the LightGBM model or exploring additional relevant features (e.g., seasonal components, external factors) to potentially improve forecasting accuracy.
